<div style="text-align: right">INFO 6105 Data Science Eng Methods and Tools, Lecture 6 Day 2</div>
<div style="text-align: right">Dino Konstantopoulos, 12 February 2020, with material by Ankur Ankan and Abinash Panda</div>

Some people have this to say about advantages of the **German** language:

<br />
<center>
<img src="ipynb.images/german-flag.jpg" width=300 />
</center>

- It is better to keep the most important piece of information at the end, to keep people’s attention. In German, the main verb in conjugation is at the end of a sentence: *Sie (You) haben (have) bestimmt (definitely) noch (still) nicht (not) viele (many) anständige (respectable) Zauberer (wizard) **kennen gelernt** (met)*. In Spanish and English, we say all the important information first and, for this reason, we tend to interrupt each other in the middle of a sentence.


- The purpose of words is to transmit **knowledge**, so they should be easily understood. Some people seem to use words no one knows just to look smart. In German, it is almost impossible to do this as names for objects describe those objects. I really love this about German: Glutenunverträglichkeit means gluten-not-compatible (celiac). It helped a lot while reading Harry Potter: Zauberer (wizard), Zauberwort (Magic word), Zaubererschule (School of Magic), Zauberstab (magic wand), Zaubererwelt (wizarding world), etc

# POS tagging with Hidden Markov Models

In this notebook you will witness how you can *cheat* Science by relying on data probabilities instead of trying to figure out the rules or laws of Science. I don't know the internals of [Universal Dependencies](https://universaldependencies.org/), but I suspect they do not worry about analyzing the structure of the German language, figuring out that the verb is at the end of a sentence, and accomodating for this in the German Tree Bank. Instead, their algorithms probably read in a lot of german text, and just by looking at the probabilities of where the verb lands in a sentence they can correctly figure out that it's at the end. Probabilities powers **statistics**, and having lots of data means your probabilities can be very *exact*.

Not too many weeks ago, you called R libraries to do POS tagging for you. Now that you know everything about probabilities, *you* can do the same thing *on your own*!

We'll use the [Brown]() corpus to build a [POS tagger](https://en.wikipedia.org/wiki/Part-of-speech_tagging), first using a simple [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) model (***most probable POS by count***), then using a **Hidden Markov Model** (HMM) that gets *transition* and *emission* probabilities from [POS bigrams](https://en.wikipedia.org/wiki/Bigram) (given a POS, what's the most probable ***next*** POS in the sentence?).

We'll divide the Brown corpus into training and test sets, and compare accuraces for BOW and HMM models.

We'll use some advanced python structures that are often used in Natural Language Processing (NLP).

# Reading in the Brown corpus efficiently

In [65]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict, namedtuple, OrderedDict
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import os
from io import BytesIO
from itertools import chain
import random

Some advanced python-fu:

Library `itertools` is a library of efficient iterators. `chain` makes an iterator that returns elements from the first iterable until it is exhausted, then proceeds to the next iterable, until all of the iterables are exhausted. It is used for treating consecutive sequences as a single sequence

In python, a single star `*` unpacks the sequence/collection into positional arguments, so you can do this:
```(python)
def sum(a, b):
    return a + b

values = (1, 2)

s = sum(*values)
```

This will unpack the tuple so that it actually executes as:
```(python)
s = sum(1, 2)
```

The double star `**` does the same, only using a dictionary and thus named arguments:
```(python)
values = { 'a': 1, 'b': 2 }
s = sum(**values)
```

A python `frozenset` is just an immutable version of a Python set object. 

While elements of a set can be modified at any time, elements of frozen set remains the same after creation. 

So, frozen sets can be used as keys in a sictionary or as element of another set.

`read_data` below reads files page by page (`\n\n`), then line by line (`\n`), uses the first line of a page as a key to an ordered dictionary, with the values being a zipper made out of words and POS tags. It accomodates the syntax of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus), pictured here below.

<br />
<center>
<img src="ipynb.images/brown.png" width=300 />
    Header of Brown corpus
</center>

In [66]:
def read_data(filename):
    """Read tagged sentence data"""
    with open(filename, 'r',encoding="UTF-8") as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
    return OrderedDict(((s[0], Sentence(*zip(*[l.strip().split("\t")
                        for l in s[1:]]))) for s in sentence_lines if s[0]))

def read_tags(filename):
    """Read a list of word tag classes"""
    with open(filename, 'r') as f:
        tags = f.read().split("\n")
    return frozenset(tags)

Sentence = namedtuple("Sentence", "words tags")

Let's read in the Brown corpus to see if our python code works out:

In [67]:
tagset = read_tags("data/brown/aa.txt")
sentences = read_data("data/brown/bb.txt")
sentences

OrderedDict([('\ufeff1',
              Sentence(words=('我们的', '班', '有', '一个', '很', '优秀的', '同学', '，', '她', '不光', '学习', '是', '我们', '班的', '第一名', '，', '而且', '在', '我们', '班', '还是', '经常', '受', '表扬的', '人物', '，', '她', '就是', '我们', '班的', '夏宇', '，', '她', '是', '一个', '特别', '优秀的', '学生', '，', '每年的', '三好', '学生', '，', '她', '还', '很', '热爱', '班', '集体'), tags=('our', 'class', 'have', 'a', 'very', 'good', 'classmate', ',', 'she', 'not only', 'study', 'is', 'our', 'class', 'first-class', ',', 'and', 'in', 'our', 'class', 'is', 'usually', 'been', 'Praise', 'character', ',', 'she', 'is', 'our', 'class', 'xiayu', ',', 'she', 'is', 'a', 'very', 'good', 'student', ',', 'annual', 'good', 'student', ',', 'she', 'also', 'very', 'like', 'class', 'collective'))),
             ('2',
              Sentence(words=('昨天', '放学的', '时候', '，', '同学们', '听到', '铃声', '，', '一窝蜂的', '都', '出去', '，', '她', '忽然', '停下', '脚步', '，', '我', '顺着', '她的', '目光', '，', '看见', '教室', '里', '非常', '杂乱', '，', '地上', '还有', '一大堆', '同学', '留下的', '纸屑'), tags=('yes

The class `Dataset` below incorporates our function above, reads in the Brown corpus and creates a collection of `keys`, a set of (unique) words, a sequence of words and a mirror sequence of tags as tuples, with `N` being the number of words in the Brown corpus.

Then it splits all this nice data into a training and test decomposition by using the `Subset` class defined further below, which mirrors the `Dataset` class.

In [68]:
class Dataset(namedtuple("_Dataset", "sentences keys vocab X tagset Y training_set testing_set N stream")):
    def __new__(cls, tagfile, datafile, train_test_split=0.8, seed=112890):
        tagset = read_tags(tagfile)
        sentences = read_data(datafile)
        keys = tuple(sentences.keys())
        wordset = frozenset(chain(*[s.words for s in sentences.values()]))
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        N = sum(1 for _ in chain(*(s.words for s in sentences.values())))
        
        # split data into train/test sets
        _keys = list(keys)
        if seed is not None: random.seed(seed)
        random.shuffle(_keys)
        split = int(train_test_split * len(_keys))
        training_data = Subset(sentences, _keys[:split])
        testing_data = Subset(sentences, _keys[split:])
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, dict(sentences), keys, wordset, word_sequences, tagset,
                               tag_sequences, training_data, testing_data, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())
    
    
class Subset(namedtuple("BaseSet", "sentences keys vocab X tagset Y N stream")):
    def __new__(cls, sentences, keys):
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        wordset = frozenset(chain(*word_sequences))
        tagset = frozenset(chain(*tag_sequences))
        N = sum(1 for _ in chain(*(sentences[k].words for k in keys)))
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, {k: sentences[k] for k in keys}, keys, wordset, word_sequences,
                               tagset, tag_sequences, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())

Let's read in the Brown corpus *again*, leveraging our classes above now, which order the corpus into efficiently navigable structures:

In [69]:
data = Dataset("data/brown/tags-universal.txt", "data/brown/bb.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 18 sentences in the corpus.
There are 14 sentences in the training set.
There are 4 sentences in the testing set.


In [70]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 256 samples of 145 unique words in the corpus.
There are 180 samples of 115 unique words in the training set.
There are 76 samples of 50 unique words in the testing set.
There are 30 words in the test set that are missing in the training set.


Let's look at an example POS tagging:

This is how easy it is, now, to access words and associated tags, using the vocabulary of Machine Learning: `X` is the independent variable, and `Y` the dependent variable!

In [71]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('我们的', '班', '有', '一个', '很', '优秀的', '同学', '，', '她', '不光', '学习', '是', '我们', '班的', '第一名', '，', '而且', '在', '我们', '班', '还是', '经常', '受', '表扬的', '人物', '，', '她', '就是', '我们', '班的', '夏宇', '，', '她', '是', '一个', '特别', '优秀的', '学生', '，', '每年的', '三好', '学生', '，', '她', '还', '很', '热爱', '班', '集体')

Labels 1: ('our', 'class', 'have', 'a', 'very', 'good', 'classmate', ',', 'she', 'not only', 'study', 'is', 'our', 'class', 'first-class', ',', 'and', 'in', 'our', 'class', 'is', 'usually', 'been', 'Praise', 'character', ',', 'she', 'is', 'our', 'class', 'xiayu', ',', 'she', 'is', 'a', 'very', 'good', 'student', ',', 'annual', 'good', 'student', ',', 'she', 'also', 'very', 'like', 'class', 'collective')

Sentence 2: ('昨天', '放学的', '时候', '，', '同学们', '听到', '铃声', '，', '一窝蜂的', '都', '出去', '，', '她', '忽然', '停下', '脚步', '，', '我', '顺着', '她的', '目光', '，', '看见', '教室', '里', '非常', '杂乱', '，', '地上', '还有', '一大堆', '同学', '留下的', '纸屑')

Labels 2: ('yesterday', 'after school', 'time', ',', 'student', 'hear', 'ring bell', 

Use `Dataset.stream()` to enumerate (word, tag) samples for the entire corpus. Let's enumerate the first 4:

In [72]:
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 3: break


Stream (word, tag) pairs:

	 ('我们的', 'our')
	 ('班', 'class')
	 ('有', 'have')
	 ('一个', 'a')
	 ('很', 'very')


These are all words and tags in our **training set**. Let's uncover the first 4:

In [110]:
words = [word for i, (word, tag) in enumerate(data.training_set.stream())]
tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
words[0:4], tags[0:4]

(['过了', '一会儿', '，', '终于'], ['after', 'a while', ',', 'finally'])

# POS Tagger using BOW Model

Let's create a dictionary of word + tag pairs where the values are just counts. Note that some words may be associated with different POS tags, in which case they will produce *distinct* pairs: 

In [111]:
def pair_counts(tags, words):
    d = defaultdict(lambda: defaultdict(int))
    for tag, word in zip(tags, words):
        d[tag][word] += 1
    return d
        
word_counts = pair_counts(words, tags)

Let's produce a dictionary where words (keys) are associated with their ***most frequent*** POS tag:

In [112]:
mfc_table = dict((word, max(tags.keys(), key=lambda key: tags[key])) for word, tags in word_counts.items())

In [113]:
i = 0
for key, value in mfc_table.items():
    print(key, value)
    i += 1
    if i > 3: break

过了 after
一会儿 a while
， ,
终于 finally


Python `namedtuple` supports a type of container-like dictionary that, like dictionaries, contains keys that are hashed to particular values. But it supports *both* access from key values as well as *iteration*, the functionality that dictionaries lack.

Let's write a class that takes in a table in its constructor and adds `<MISSING>` POS tags if the word is missing from the training set (possible that a word is in the test set but missing from the training set). It also has a `viterbi` method that takes in the table and builds a sequence of states that we will use in our Hidden Markov Model.

In [114]:
FakeState = namedtuple('FakeState', 'name')

class MFCTagger:
    missing = FakeState(name = '<MISSING>')
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))

In [115]:
mfc_model = MFCTagger(mfc_table)

So essentially we built a table that associates words with their most frequent POS tag. This is a simplistic **bag of words** (BOW) model. Let's see, given a sentence, if we *guess the hidden states* (POS tags) right!

In [116]:
def replace_unknown(sequence):
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):    
    _, state_path = model.viterbi(replace_unknown(X)) 
    return [state[1].name for state in state_path[1:-1]]
   

In [117]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Sentence: {}\n".format(data.sentences[key].words))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: 12

Sentence: ('说完', '我', '就去', '买', '东西了')

Predicted labels:
-----------------
['<MISSING>', 'I', '<MISSING>', 'buy', '<MISSING>']

Actual labels:
--------------
('after that', 'I', 'just', 'go', 'shopping')


Sentence Key: 6

Sentence: ('夏宇', '把', '垃圾', '装在', '一个', '袋子', '，', '和', '我', '走出', '教室', '，', '关好', '门', '，', '去', '扔', '垃圾')

Predicted labels:
-----------------
['<MISSING>', 'hold', 'garbage', '<MISSING>', '<MISSING>', '<MISSING>', ',', 'and', 'I', '<MISSING>', 'class', ',', '<MISSING>', '<MISSING>', ',', 'go', '<MISSING>', 'garbage']

Actual labels:
--------------
('xiayu', 'place', 'garbage', 'in', 'a', 'bag', ',', 'with', 'me', 'go out', 'class', ',', 'close', 'door', ',', 'go', 'throw', 'garbage')


Sentence Key: ﻿1

Sentence: ('我们的', '班', '有', '一个', '很', '优秀的', '同学', '，', '她', '不光', '学习', '是', '我们', '班的', '第一名', '，', '而且', '在', '我们', '班', '还是', '经常', '受', '表扬的', '人物', '，', '她', '就是', '我们', '班的', '夏宇', '，', '她', '是', '一个', '特别', '优秀的', '学生', '，', '每年的', '三

Pretty good! Let's evaluate the accuracy of our most-frequent-tag tagger:

In [118]:
def accuracy(X, Y, model):
    
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [119]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

training accuracy mfc_model: 95.00%
testing accuracy mfc_model: 40.79%


# Hidden Markov Model

Let's build a POS tagger using a Hidden Markov Model.

First, let's see how many POS tags we have in our corpus, using the python `Counter` structure that we used last week to count instances.

In [83]:
def unigram_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
tag_unigrams = unigram_counts(tags)
tag_unigrams

Counter({'after': 1,
         'a while': 1,
         ',': 25,
         'finally': 1,
         'clean': 3,
         'finish': 1,
         'she': 5,
         'because': 2,
         'run': 1,
         'too': 1,
         'fast': 1,
         'incaution': 1,
         'fell': 1,
         'this': 1,
         'is': 2,
         'our': 4,
         'class': 4,
         'good': 1,
         'student': 3,
         'I': 11,
         'also': 3,
         'from': 1,
         'her': 1,
         'study': 2,
         'not only': 1,
         'want': 1,
         'well': 1,
         'more': 1,
         'care': 1,
         'collective': 2,
         'home': 2,
         'said': 2,
         'okay': 1,
         'uncle': 1,
         'love': 4,
         'eating': 4,
         'apples': 2,
         'apple': 2,
         'one day': 1,
         'mom': 2,
         'let': 1,
         'me': 1,
         'go': 1,
         'buy': 1,
         'something': 1,
         'happily': 1,
         'Okay': 1,
         'know': 1,
        

We'll *slightly* modify the code above to get our POS bigrams, from *both* training and test subsets, to uncover which POS tags follow which other POS tags. So, instead of a simple list of POS tags, `Counter` will count *neighboring* POS tuples! 

In [84]:
def bigram_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.stream())]
o = [(tags[i],tags[i+1]) for i in range(0,len(tags)-2,2)]
tag_bigrams = bigram_counts(o)
tag_bigrams 

Counter({('our', 'class'): 4,
         ('have', 'a'): 1,
         ('very', 'good'): 1,
         ('classmate', ','): 1,
         ('she', 'not only'): 1,
         ('study', 'is'): 1,
         ('first-class', ','): 1,
         ('and', 'in'): 1,
         ('is', 'usually'): 1,
         ('been', 'Praise'): 1,
         ('character', ','): 1,
         ('she', 'is'): 2,
         ('xiayu', ','): 1,
         ('a', 'very'): 1,
         ('good', 'student'): 2,
         (',', 'annual'): 1,
         (',', 'she'): 2,
         ('also', 'very'): 1,
         ('like', 'class'): 1,
         ('collective', 'yesterday'): 1,
         ('after school', 'time'): 1,
         (',', 'student'): 1,
         ('hear', 'ring bell'): 1,
         (',', 'a lot'): 1,
         ('all', 'get out'): 1,
         ('suddently', 'stop'): 1,
         ('pace', ','): 1,
         ('I', 'along'): 1,
         ('hers', 'eye'): 1,
         (',', 'see'): 1,
         ('class', 'in'): 1,
         ('very', 'massly'): 1,
         (',', 'ground

What tags do our sentences *begin* with?

In [85]:
def starting_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.stream())]
starts_tag = [i[0] for i in data.Y]
tag_starts = starting_counts(starts_tag)
tag_starts

Counter({'our': 1,
         'yesterday': 1,
         'we': 1,
         'after': 1,
         'xiayu': 1,
         'she': 1,
         'I': 8,
         'this': 1,
         'march': 1,
         'one day': 1,
         'after that': 1})

What tags do our sentences *end* with?

In [86]:
def ending_counts(sequences):    
    return Counter(sequences)

end_tag = [i[len(i)-1] for i in data.Y]
tag_ends = ending_counts(end_tag)
tag_ends

Counter({'collective': 1,
         'paper': 1,
         'busy': 1,
         'finish': 1,
         'garbage': 1,
         'fell': 1,
         'home': 2,
         'walking': 1,
         'know': 1,
         'shopping': 1,
         'uncle': 1,
         'apple': 2,
         'pears': 2,
         'apples': 2})

Not surprising that most end with a period `.`! Ideally, we should end with the previous-to-last tag! 

In [87]:
end_tag = [i[len(i)-2] for i in data.Y]
tag_ends = ending_counts(end_tag)
tag_ends

Counter({'class': 1,
         'left': 1,
         'very': 1,
         'clean': 1,
         'throw': 1,
         'incaution': 1,
         'back': 1,
         'our': 1,
         'shyly': 1,
         'I': 1,
         'go': 1,
         'okay': 1,
         'one': 1,
         'eating': 4,
         'and': 1})

Let's create our Hidden Markov Model and peek into most popular words per POS tag.

`tag_words_count` contains words associated to each POS tag, arranged by frequency so that we can eventually evaluate *emission* probabilities, which are probabilities of observable states (wrods) given hidden states (POS tags).

In [88]:
hmm_model = HiddenMarkovModel(name="base-hmm-tagger")

tags = [tag for i, (word, tag) in enumerate(data.stream())]
words = [word for i, (word, tag) in enumerate(data.stream())]

tags_count = unigram_counts(tags)
tag_words_count = pair_counts(tags, words)

starting_tag_list = [i[0] for i in data.Y]
#ending_tag_list = [i[-1] if len(i)==1 else i[-2] for i in data.Y]
#ending_tag_list = [i[-1] for i in data.Y]
ending_tag_list = [i[len(i)-2] for i in data.Y]

starting_tag_count = starting_counts(starting_tag_list) #the number of times a tag occured at the start
ending_tag_count = ending_counts(ending_tag_list)       #the number of times a tag occured at the end

tag_words_count

defaultdict(<function __main__.pair_counts.<locals>.<lambda>()>,
            {'our': defaultdict(int, {'我们的': 4, '我们': 4}),
             'class': defaultdict(int, {'班': 5, '班的': 3, '教室': 2}),
             'have': defaultdict(int, {'有': 1}),
             'a': defaultdict(int, {'一个': 3}),
             'very': defaultdict(int, {'很': 2, '特别': 1, '非常': 2}),
             'good': defaultdict(int, {'优秀的': 2, '三好': 1, '优秀': 1}),
             'classmate': defaultdict(int, {'同学': 1}),
             ',': defaultdict(int, {'，': 31, '、': 2, ',': 1}),
             'she': defaultdict(int, {'她': 9}),
             'not only': defaultdict(int, {'不光': 1, '不仅': 1}),
             'study': defaultdict(int, {'学习': 3}),
             'is': defaultdict(int, {'是': 2, '还是': 1, '就是': 3}),
             'first-class': defaultdict(int, {'第一名': 1}),
             'and': defaultdict(int, {'而且': 1, '和': 1}),
             'in': defaultdict(int, {'在': 1, '里': 1, '装在': 1}),
             'usually': defaultdict(int, {'经常': 1}),

In [89]:
ending_tag_list

['class',
 'left',
 'very',
 'clean',
 'throw',
 'incaution',
 'back',
 'our',
 'shyly',
 'I',
 'go',
 'okay',
 'one',
 'eating',
 'and',
 'eating',
 'eating',
 'eating']

Let's convert word frequencies by POS tag to probabilities by dividing by the total number of words per POS tag, yielding the `distribution` of words.

We'll define HMM emission probabilities using that `distribution`.

In [90]:
tag_words_count

defaultdict(<function __main__.pair_counts.<locals>.<lambda>()>,
            {'our': defaultdict(int, {'我们的': 4, '我们': 4}),
             'class': defaultdict(int, {'班': 5, '班的': 3, '教室': 2}),
             'have': defaultdict(int, {'有': 1}),
             'a': defaultdict(int, {'一个': 3}),
             'very': defaultdict(int, {'很': 2, '特别': 1, '非常': 2}),
             'good': defaultdict(int, {'优秀的': 2, '三好': 1, '优秀': 1}),
             'classmate': defaultdict(int, {'同学': 1}),
             ',': defaultdict(int, {'，': 31, '、': 2, ',': 1}),
             'she': defaultdict(int, {'她': 9}),
             'not only': defaultdict(int, {'不光': 1, '不仅': 1}),
             'study': defaultdict(int, {'学习': 3}),
             'is': defaultdict(int, {'是': 2, '还是': 1, '就是': 3}),
             'first-class': defaultdict(int, {'第一名': 1}),
             'and': defaultdict(int, {'而且': 1, '和': 1}),
             'in': defaultdict(int, {'在': 1, '里': 1, '装在': 1}),
             'usually': defaultdict(int, {'经常': 1}),

In [91]:
to_pass_states = []
for tag, words_dict in tag_words_count.items():
    total = float(sum(words_dict.values()))
    
    distribution = {word: count/total for word, count in words_dict.items()}
    print(distribution)
    tag_emissions = DiscreteDistribution(distribution)
    tag_state = State(tag_emissions, name=tag)
    to_pass_states.append(tag_state)

{'我们的': 0.5, '我们': 0.5}
{'班': 0.5, '班的': 0.3, '教室': 0.2}
{'有': 1.0}
{'一个': 1.0}
{'很': 0.4, '特别': 0.2, '非常': 0.4}
{'优秀的': 0.5, '三好': 0.25, '优秀': 0.25}
{'同学': 1.0}
{'，': 0.9117647058823529, '、': 0.058823529411764705, ',': 0.029411764705882353}
{'她': 1.0}
{'不光': 0.5, '不仅': 0.5}
{'学习': 1.0}
{'是': 0.3333333333333333, '还是': 0.16666666666666666, '就是': 0.5}
{'第一名': 1.0}
{'而且': 0.5, '和': 0.5}
{'在': 0.3333333333333333, '里': 0.3333333333333333, '装在': 0.3333333333333333}
{'经常': 1.0}
{'受': 1.0}
{'表扬的': 1.0}
{'人物': 1.0}
{'夏宇': 1.0}
{'学生': 0.4, '同学们': 0.2, '同学': 0.4}
{'每年的': 1.0}
{'还': 0.25, '还有': 0.25, '也要': 0.25, '也': 0.25}
{'热爱': 1.0}
{'集体': 1.0}
{'昨天': 1.0}
{'放学的': 1.0}
{'时候': 1.0}
{'听到': 1.0}
{'铃声': 1.0}
{'一窝蜂的': 1.0}
{'都': 1.0}
{'出去': 1.0}
{'忽然': 1.0}
{'停下': 1.0}
{'脚步': 1.0}
{'我': 1.0}
{'顺着': 1.0}
{'她的': 1.0}
{'目光': 1.0}
{'看见': 1.0}
{'杂乱': 1.0}
{'地上': 1.0}
{'一大堆': 1.0}
{'留下的': 1.0}
{'纸屑': 1.0}
{'我们': 1.0}
{'扫地': 0.3333333333333333, '打扫': 0.3333333333333333, '收拾': 0.3333333333333333}
{'摆': 0.5, 

In [92]:
distribution

{'苹果': 0.6666666666666666, '水果': 0.3333333333333333}

`to_pass_states` yields the probability distribution of words per POS tag:

In [93]:
to_pass_states

[{
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "\u6211\u4eec\u7684" : 0.5,
                 "\u6211\u4eec" : 0.5
             }
         ],
         "frozen" : false
     },
     "name" : "our",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "\u73ed" : 0.5,
                 "\u73ed\u7684" : 0.3,
                 "\u6559\u5ba4" : 0.2
             }
         ],
         "frozen" : false
     },
     "name" : "class",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
          

Let's add states to our model:

In [94]:
hmm_model.add_states() 

The start probability for each tag is how many times it is a sentence-starting POS tag divided by its total count. We build the starting transitions for our HMM model:

In [95]:
start_prob={}

for tag in tags:
    start_prob[tag] = starting_tag_count[tag] / tags_count[tag]

for tag_state in to_pass_states :
    hmm_model.add_transition(hmm_model.start, tag_state, start_prob[tag_state.name])  

The end probability for each tag is how many times it is a sentence-ending POS tag divided by its total count. We build the ending transitions for our HMM model:

In [96]:
end_prob={}

for tag in tags:
    end_prob[tag] = ending_tag_count[tag]/tags_count[tag]
    
for tag_state in to_pass_states :
    hmm_model.add_transition(tag_state, hmm_model.end, end_prob[tag_state.name])
    print(tag_state)

{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "\u6211\u4eec\u7684" : 0.5,
                "\u6211\u4eec" : 0.5
            }
        ],
        "frozen" : false
    },
    "name" : "our",
    "weight" : 1.0
}
{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "\u73ed" : 0.5,
                "\u73ed\u7684" : 0.3,
                "\u6559\u5ba4" : 0.2
            }
        ],
        "frozen" : false
    },
    "name" : "class",
    "weight" : 1.0
}
{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "\u6709" : 1.0
            }
        

We now add the transition probabilities for our model, which uses our POS bigrams to enumerate what the probabilities are for transiting from one POS tag to another.

In [120]:
transition_prob_pair={}

for key in tag_bigrams.keys():
    print(key)
    transition_prob_pair[key] = tag_bigrams.get(key)/tags_count[key[0]]
    
    
for tag_state in to_pass_states :
    for next_tag_state in to_pass_states:
        #hmm_model.add_transition(tag_state, next_tag_state,0)
        if (tag_state.name, next_tag_state.name) not in transition_prob_pair:
            transition_prob_pair[(tag_state.name, next_tag_state.name)] = 0
        hmm_model.add_transition(tag_state, next_tag_state, transition_prob_pair[(tag_state.name, next_tag_state.name)])
       

('our', 'class')
('have', 'a')
('very', 'good')
('classmate', ',')
('she', 'not only')
('study', 'is')
('first-class', ',')
('and', 'in')
('is', 'usually')
('been', 'Praise')
('character', ',')
('she', 'is')
('xiayu', ',')
('a', 'very')
('good', 'student')
(',', 'annual')
(',', 'she')
('also', 'very')
('like', 'class')
('collective', 'yesterday')
('after school', 'time')
(',', 'student')
('hear', 'ring bell')
(',', 'a lot')
('all', 'get out')
('suddently', 'stop')
('pace', ',')
('I', 'along')
('hers', 'eye')
(',', 'see')
('class', 'in')
('very', 'massly')
(',', 'ground')
('also', 'a load of')
('student', 'left')
('paper', 'we')
('clean', ',')
('place', 'table')
(',', 'wipe')
('blackboard', ',')
('very', 'busy')
('after', 'a while')
(',', 'finally')
('clean', 'finish')
('xiayu', 'place')
('garbage', 'in')
('a', 'bag')
(',', 'with')
('me', 'go out')
('class', ',')
('close', 'door')
(',', 'go')
('throw', 'garbage')
('she', 'because')
('run', 'too')
('fast', ',')
('incaution', 'fell')
('I'

We *bake* our model:

In [121]:
hmm_model.bake()

KeyboardInterrupt: 

We can now evaluate the accuracy of our HMM model and compare it to our BOW model:

In [ ]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, hmm_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, hmm_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

Here's a decoding example:

In [ ]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    #print(type(data.sentences[key]))
    print("Sentence: {}\n".format(data.sentences[key].words))
    print("Predicted labels:\n-----------------")
    #print(data.sentences[key].words)
    print(simplify_decoding(data.sentences[key].words, hmm_model))
    
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

A 96% accuracy for our HMM model compared to a 93% accuracy for our BOW model is a ***huge*** improvement as it brings language understanding error to below 4%, and 5% error is considered a *gold standard* for NLP. Speech-to-text frameworks like Alexa and Siri only started betting popular when they crossed the 5% threshold.

# References

- Hands on Markov models with python, by Ankur Ankan and Abinash Panda, [on amazon](https://www.amazon.com/Hands-Markov-Models-Python-probabilistic/dp/1788625447/ref=sr_1_2?keywords=hands+on+markov+models+with+python&qid=1581280984&sr=8-2)</div>

- [Universal Dependency Parsing from Scratch](https://nlp.stanford.edu/pubs/qi2018universal.pdf)

- [Statistical Machine Translation](https://en.wikipedia.org/wiki/Statistical_machine_translation)

- [Language Models](https://en.wikipedia.org/wiki/Language_model).

# Homework

Use the methodology in this notebook to build a statistical language translator, *from your language to english*. So, from Hindi or Chinese to English. Teams of **3** students. You *have* to use a Hidden Markov Model and `pomegranate` as your HMM library, to ensure all student teams start from the same baseline. Start from a Most Frequent Word (BOW) translation baseline, then move on to a Hidden Markov Model to improve translation. How much can you improve it by? The translation engine with the best accuracy ***improvement***, per language, will be presented in class.